## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Torbay,CA,47.6666,-52.7314,30.27,87,100,10.36,light snow
1,1,Kutum,SD,14.2000,24.6667,55.76,24,3,6.26,clear sky
2,2,Yeppoon,AU,-23.1333,150.7333,85.78,73,40,20.31,scattered clouds
3,3,Mataura,NZ,-46.1927,168.8643,64.47,39,100,5.77,overcast clouds
4,4,Vaini,TO,-21.2000,-175.2000,84.36,84,40,10.36,scattered clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Yeppoon,AU,-23.1333,150.7333,85.78,73,40,20.31,scattered clouds
4,4,Vaini,TO,-21.2000,-175.2000,84.36,84,40,10.36,scattered clouds
6,6,Butaritari,KI,3.0707,172.7902,81.28,77,67,12.15,light rain
12,12,Saint-Pierre,RE,-21.3393,55.4781,82.08,74,0,2.30,clear sky
14,14,Isangel,VU,-19.5500,169.2667,87.91,74,40,6.91,scattered clouds
26,26,Maragogi,BR,-9.0122,-35.2225,78.35,76,55,8.03,broken clouds
34,34,Avarua,CK,-21.2078,-159.7750,75.25,94,40,9.22,scattered clouds
37,37,Busselton,AU,-33.6500,115.3333,91.04,40,0,0.83,clear sky
38,38,Kandrian,PG,-6.2167,149.5500,82.35,80,88,4.03,moderate rain
41,41,Atuona,PF,-9.8000,-139.0333,77.31,80,22,13.73,light rain


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                163
City                   163
Country                162
Lat                    163
Lng                    163
Max Temp               163
Humidity               163
Cloudiness             163
Wind Speed             163
Current Description    163
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace("", np.nan, inplace=True)
preferred_cities_df.dropna(inplace=True)

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yeppoon,AU,85.78,scattered clouds,-23.1333,150.7333,
4,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,
6,Butaritari,KI,81.28,light rain,3.0707,172.7902,
12,Saint-Pierre,RE,82.08,clear sky,-21.3393,55.4781,
14,Isangel,VU,87.91,scattered clouds,-19.5500,169.2667,
26,Maragogi,BR,78.35,broken clouds,-9.0122,-35.2225,
34,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
37,Busselton,AU,91.04,clear sky,-33.6500,115.3333,
38,Kandrian,PG,82.35,moderate rain,-6.2167,149.5500,
41,Atuona,PF,77.31,light rain,-9.8000,-139.0333,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): 
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [30]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yeppoon,AU,85.78,scattered clouds,-23.1333,150.7333,Villa Mar Colina
4,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Butaritari,KI,81.28,light rain,3.0707,172.7902,Isles Sunset Lodge
12,Saint-Pierre,RE,82.08,clear sky,-21.3393,55.4781,Lindsey Hôtel
14,Isangel,VU,87.91,scattered clouds,-19.5500,169.2667,Tanna Lodge
...,...,...,...,...,...,...,...
663,Launceston,AU,87.62,overcast clouds,-41.4500,147.1667,Hotel Grand Chancellor Launceston
670,Voh,NC,86.99,overcast clouds,-20.9667,164.7000,Le gîte du Koniambo
674,Ugoofaaru,MV,81.30,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
685,Bubaque,GW,77.94,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df = hotel_df
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))